# Extract Data from Schools

In [1]:
# Global Packages
import requests
import certifi
from time import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


### Parameters

In [2]:
BASE_URL = "https://www.redacademica.edu.co/colegios?name=&field_localidad_target_id=All"
HEADERS = { "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
           "Host": "www.redacademica.edu.co" }

In [21]:
# find all a's with href
def get_urls_dict(soup):
    """
    Getting school URLS from the given soup object
    """
    urls_dict = {}
    a_objects = soup.find_all("a", href=True)
    for a_object in a_objects:
        if "colegios/" in str.lower(a_object["href"]):
            new_url = "https://www.redacademica.edu.co" + a_object["href"]
            urls_dict[a_object.text] = new_url

    if "\n\n\n\n" in urls_dict:
        del urls_dict["\n\n\n\n"]

    return urls_dict

### Playground

In [3]:
r = requests.get(BASE_URL, headers=HEADERS)
soup = BeautifulSoup(r.content, "html.parser")

In [23]:
urls_dict = get_urls_dict(soup)
urls_dict

{'Colegio Aquileo Parra': 'https://www.redacademica.edu.co/colegios/colegio-aquileo-parra',
 'IED Colegio El Verjon': 'https://www.redacademica.edu.co/colegios/ied-colegio-el-verjon',
 'Colegio Altamira Sur Oriental (IED)': 'https://www.redacademica.edu.co/colegios/colegio-altamira-sur-oriental-ied',
 'Colegio Montebello (IED)': 'https://www.redacademica.edu.co/colegios/colegio-montebello-ied',
 'Colegio Atenas (IED)': 'https://www.redacademica.edu.co/colegios/colegio-atenas-ied',
 'Colegio Jose Joaquin Castro Martinez (IED)': 'https://www.redacademica.edu.co/colegios/colegio-jose-joaquin-castro-martinez-ied',
 'Colegio Entre Nubes Sur Oriental (IED)': 'https://www.redacademica.edu.co/colegios/colegio-entre-nubes-sur-oriental-ied',
 'Colegio Diego Montaña Cuellar (IED)': 'https://www.redacademica.edu.co/colegios/colegio-diego-montana-cuellar-ied',
 'Colegio Gran Yomasa I. E. D.': 'https://www.redacademica.edu.co/colegios/colegio-gran-yomasa-i-e-d',
 'Colegio Marco Fidel Suárez (IED)': 

In [25]:
# Getting info from first school
school_name = list(urls_dict.keys())[0]
school_url = urls_dict[school_name]
school_r = requests.get(school_url, headers=HEADERS)
school_soup = BeautifulSoup(school_r.content, "html.parser")

In [26]:
info_container = school_soup.find("ul", {"class": "school-info__list"})
info_container

<ul class="school-info__list">
<li class="school-info__list-item">
<h4>Jornada</h4>
<p>
<div class="field field--name-field-jornada field--type-list-string field--label-hidden field__items">
<div class="field__item">Mañana</div>
<div class="field__item">Tarde</div>
<div class="field__item">Nocturna</div>
</div>
</p>
</li>
<li class="school-info__list-item">
<h4>Dirección</h4>
<p>Carrera 18A # 187-75</p>
</li>
<li class="school-info__list-item">
<h4>Correo</h4>
<a href="mailto:insdiaquileoparrav1@educacionbogota.edu.co">insdiaquileoparrav1@educacionbogota.edu.co</a>
</li>
<li class="school-info__list-item">
<h4>Teléfono</h4>
<a href="tel:6745973-6745973">6745973 - 6745973</a>
</li>
<li class="school-info__list-item">
<h4>Horario de Atención</h4>
<p>8:00 a.m a 4:00 p.m.</p>
</li>
</ul>